# GSM8K: Advanced DSPy Techniques with Local Models

This notebook demonstrates advanced DSPy optimization techniques using a local Ollama model (lfm2.5-thinking:latest) on math word problems.

**Goal**: Explore cutting-edge DSPy features to maximize performance with local models.

## What We'll Cover

- Advanced DSPy optimizers (MIPRO)
- Chain-of-thought prompting
- Multi-stage optimization
- Local model integration with Ollama
- Performance analysis and comparison

## Prerequisites

- Ollama installed and running
- `lfm2.5-thinking:latest` model pulled: `ollama pull lfm2.5-thinking:latest`
- OpenAI API key (for comparison if needed)

## 1. Setup

In [5]:
import sys
import os
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

import dspy
# MIPRO may not be available in the installed dspy package — fall back to the
# local wrapper provided in this repository when necessary.
try:
    from dspy.teleprompt import MIPRO, BootstrapFewShotWithRandomSearch
except Exception:
    from optimizers.dspy_optimizers import MIPROOptimizer as MIPRO
    from dspy.teleprompt import BootstrapFewShotWithRandomSearch

# Some dspy versions don't export convenience primitives used in older notebooks;
# provide lightweight fallbacks so the notebook runs across dspy releases.
try:
    from dspy.primitives import assert_transform_module, suggest_transform_module
except Exception:
    def assert_transform_module(module, *args, **kwargs):
        """Fallback that assumes module is valid (used only for demos)."""
        return True

    def suggest_transform_module(module, *args, **kwargs):
        """Fallback stub returning None for suggestions."""
        return None

from data import prepare_gsm8k_splits, gsm8k_metric, evaluate_gsm8k
from modules import MathSolver
from baselines import create_baseline
from utils import Evaluator
from config import DATASET_CONFIGS

import warnings
warnings.filterwarnings('ignore')

print("✓ Advanced DSPy imports successful")

✓ Advanced DSPy imports successful


In [ ]:
# Configure Ollama model
ollama_lm = dspy.LM(
    model="ollama/lfm2.5-thinking:latest",
    api_base="http://localhost:11434/v1",
    api_key=None,  # don't send empty Bearer header to Ollama
    max_tokens=4096,
    temperature=0.7,
)

# Test the model
print("Testing Ollama model...")
try:
    response = ollama_lm("What is 2+2?")
    print(f"✓ Model response: {response}")
except Exception as e:
    print(f"❌ Model test failed: {e}")
    print("Make sure Ollama is running and the model is pulled.")

# Configure DSPy
dspy.settings.configure(lm=ollama_lm)

print("✓ Configured DSPy with Ollama lfm2.5-thinking:latest")

Testing Ollama model...
❌ Model test failed: litellm.APIConnectionError: OllamaException - Illegal header value b'Bearer '
Make sure Ollama is running and the model is pulled.
✓ Configured DSPy with Ollama lfm2.5-thinking:latest


## 2. Load Data

In [ ]:
# Load GSM8K splits
config = DATASET_CONFIGS['gsm8k'].copy()
config['dev_size'] = 100  # Smaller for faster iteration

train_examples, dev_examples, test_examples = prepare_gsm8k_splits(
    train_size=config['train_size'],
    dev_size=config['dev_size'],
    test_size=config['test_size'],
    seed=config['seed'],
)

print(f"✓ Data loaded:")
print(f"  Train: {len(train_examples)} examples")
print(f"  Dev:   {len(dev_examples)} examples")
print(f"  Test:  {len(test_examples)} examples")

## 3. Advanced DSPy Module with Chain-of-Thought

In [ ]:
# Create advanced math solver with chain-of-thought
class AdvancedMathSolver(dspy.Module):
    def __init__(self):
        super().__init__()
        self.solve = dspy.ChainOfThought("question -> reasoning, answer")
    
    def forward(self, question):
        return self.solve(question=question)

# Test the advanced module
advanced_solver = AdvancedMathSolver()
test_example = train_examples[0]

print("Testing Advanced Math Solver:")
print(f"Question: {test_example.question}")
result = advanced_solver.forward(question=test_example.question)
print(f"Reasoning: {result.reasoning}")
print(f"Answer: {result.answer}")
print(f"Expected: {test_example.answer}")

## 4. MIPRO Optimization

In [ ]:
# MIPRO (Multi-Instructor Prompt Optimization)
mipro_kwargs = dict(
    metric=gsm8k_metric,
    num_candidates=10,  # Generate 10 candidate programs
    init_temperature=1.0,  # High temperature for diverse initial prompts
)

print("Starting MIPRO optimization...")
print("This will generate multiple candidate programs and select the best one.")

# Instantiate with compatibility for either dspy.MIPRO or the local wrapper
try:
    mipro_optimizer = MIPRO(**mipro_kwargs, verbose=True, num_threads=4)
except TypeError:
    mipro_optimizer = MIPRO(**mipro_kwargs)

optimized_solver = mipro_optimizer.compile(
    AdvancedMathSolver(),
    trainset=train_examples[:30],  # Smaller training set for demo
    valset=dev_examples[:30],      # provide validation set (required by some optimizers)
    eval_kwargs=dict(num_threads=4, display_table=False),
)

print("✓ MIPRO optimization complete!")

## 5. Evaluation with Concurrent Processing

In [ ]:
# Evaluate all approaches
evaluator = Evaluator(
    metric_fn=gsm8k_metric, 
    show_progress=True, 
    verbose=False, 
    max_concurrent=5  # Concurrent evaluation
)

# Zero-shot baseline
zero_shot = create_baseline("zero-shot", "gsm8k", ollama_lm)
zero_shot_result = evaluator.evaluate(
    zero_shot, dev_examples, "Zero-Shot (Ollama)", "gsm8k"
)

# Few-shot baseline
few_shot = create_baseline("few-shot", "gsm8k", ollama_lm, num_examples=3)
few_shot_result = evaluator.evaluate(
    few_shot, dev_examples, "Few-Shot (Ollama)", "gsm8k"
)

# DSPy optimized
dspy_result = evaluator.evaluate(
    optimized_solver, dev_examples, "DSPy MIPRO (Ollama)", "gsm8k"
)

print("
📊 Results Summary:")
print(f"Zero-Shot: {zero_shot_result.accuracy:.1%}")
print(f"Few-Shot: {few_shot_result.accuracy:.1%}")
print(f"DSPy MIPRO: {dspy_result.accuracy:.1%}")

## 6. Inspect Optimized Program

In [ ]:
# Inspect what MIPRO learned
print("Optimized Program Structure:")
print(optimized_solver)

# Test on a few examples
print("\nSample Predictions:")
for i in range(3):
    example = dev_examples[i]
    pred = optimized_solver(question=example.question)
    correct = gsm8k_metric(example, pred) > 0.5
    print(f"\nExample {i+1}: {'✓' if correct else '✗'}")
    print(f"Question: {example.question[:60]}...")
    print(f"Predicted: {pred.answer}")
    print(f"Expected: {example.answer}")

## 7. Advanced DSPy Insights

### What We Learned

1. **MIPRO vs BootstrapFewShot**: MIPRO generates multiple candidate programs and uses Bayesian optimization to find the best one
2. **Chain-of-Thought**: Explicit reasoning steps help local models perform better
3. **Local Models**: Ollama models can achieve good performance with proper optimization
4. **Concurrent Processing**: Speeds up both optimization and evaluation

### Advanced Techniques Used

- **MIPRO Optimizer**: Multi-instructor prompt optimization
- **ChainOfThought Signature**: Forces step-by-step reasoning
- **Bayesian Optimization**: Efficiently searches prompt space
- **Parallel Processing**: Concurrent API calls for speed

### Future Enhancements

- Try different local models (Mixtral, Llama variants)
- Experiment with assertion-based optimization
- Add multi-stage reasoning pipelines
- Implement custom metrics and constraints